# Hit-And-Run With Sliding Trees - Top 3% (Silver) Finish

Many of the top RPS agents use relatively complex strategies and meta-strategies to select each move. It is possible, however, to use a single, basic model (i.e. no selection strategies) with a couple of other techniques and achieve a respectable score on the leaderboard, as demonstrated below.

### Bagging Classifier With Sliding Number Of Trees

Each turn, the agent attempts to predict the opponent's next move (and then plays the move which beats that move) using a standard bagging classifier trained using the history of the match. The only difference with this classifier is that the number of trees used in the model increases (hence "sliding" trees) as each match progresses and therefore more training data becomes available. In this case the number of trees starts at just 1 at the beginning of each match and increases by 1 every 50 turns. This ensures that the size of the subset of training data each tree uses remains fairly constant throughout the entire match.

### Hit-And-Run

The bagging classifier produces a probability estimate for each of the opponent's moves and using these, we can see how confident the classifier is that the opponent will play the predicted move. The agent only plays the move which beats the predicted opponent's move when the predicted probability of the opponent making that move is over 70%. The rest of the time the agent simply plays a random move. This means when it is unsure, neither the agent nor its opponent has the edge as a random move will win/draw/lose in equal amounts. It also helps to cloak the agent's strategy as it adds a lot of noise to its moves. In this sense it's a rather cowardly agent as it only "attacks" (and potentially exposes its strategy) when it thinks the odds are in its favour!

### Random Start

The agent also plays randomly at the beginning of each match, to help obfuscate its strategy. It also doesn't collect any of the opponent's moves as training data during this period in case the opponent also has a random warm-up period.

### That's It!

I wasn't going to do a write-up as there are certainly stronger agents out there. However, in the end I thought it may be worth it as many of the best ones are likely to feature relatively advanced strategy selection techniques and large ensemble methods, and I thought it would be good to show that you can create a decent contender with a fairly simple and compact agent without having to write too much code.

Thanks for reading!

## Agent Code

In [ ]:
import random
from pandas import DataFrame
from sklearn.ensemble import BaggingClassifier

numTurnsPredictors = 5 #number of previous turns to use as predictors
minTrainSetRows = 10 #only start predicting moves after we have enough data
numRandomStartTurns = 200
myLastMove = None
mySecondLastMove = None
opponentLastMove = None
numDummies = 3 #how many dummy vars we need to represent a move
predictors = DataFrame(columns=[str(x) for x in range(numTurnsPredictors * 2 * numDummies)])
predictors = predictors.astype("int")
opponentsMoves = []
roundHistory = [] #moves made by both players in each round
clf = BaggingClassifier()

def randomMove():
    return random.randint(0,2)

#converts my and opponents moves into dummy variables i.e. [1,2] into [0,1,1,0]
def convertToDummies(moves):
    newMoves = []
    dummies = [[0,0,1], [0,1,0], [1,0,0]]

    for move in moves:
        newMoves.extend(dummies[move])

    return newMoves

def updateRoundHistory(myMove, opponentMove):
    global roundHistory
    roundHistory.append(convertToDummies([myMove, opponentMove]))

def flattenData(data):
    return sum(data, [])

def updateFeatures(rounds):
    global predictors
    flattenedRounds = flattenData(rounds)
    predictors.loc[len(predictors)] = flattenedRounds

#returns index of biggest value
def getIndexMax(probs):
    return probs.index(max(probs))

#is the largest probabilty class above the threshold
def isStrongPrediction(probs):
    largestProb = probs[getIndexMax(probs)]
    
    #more than 70%
    if largestProb > 0.7:
        return True
    else:
        return False

def fitAndPredict(clf, x, y, newX):
    df = DataFrame.from_records([newX], columns=[str(i) for i in range(numTurnsPredictors * 2 * numDummies)])
    clf.fit(x, y)
    probs = clf.predict_proba(df)[0].tolist()
    print(probs)
    
    #only play non-random move if classifier is confident enough
    if isStrongPrediction(probs):
        print("PLAYING!")
        return getIndexMax(probs)
    else:
        return randomMove()

def makeMove(observation, configuration):
    global myLastMove
    global mySecondLastMove
    global opponentLastMove
    global predictors
    global opponentsMoves
    global roundHistory

    if observation.step == 0:
        myLastMove = randomMove()
        return myLastMove

    if observation.step == 1:
        updateRoundHistory(myLastMove, observation.lastOpponentAction)
        myLastMove = randomMove()
        return myLastMove

    else:
        updateRoundHistory(myLastMove, observation.lastOpponentAction)
        opponentsMoves.append(observation.lastOpponentAction)

        if observation.step > numTurnsPredictors and observation.step > numRandomStartTurns:
            updateFeatures(roundHistory[-numTurnsPredictors - 1: -1])

        if len(predictors) > minTrainSetRows:
            print("PREDICTING")
            clf = BaggingClassifier(n_estimators=max(int(observation.step/50), 1))
            predictX = flattenData(roundHistory[-numTurnsPredictors:]) #data to predict next move
            predictedMove = fitAndPredict(clf, predictors,
                              opponentsMoves[-len(predictors):], predictX)
            myLastMove = (predictedMove + 1) % 3
            return myLastMove
        else:
            myLastMove = randomMove()
            return myLastMove